In [1]:
import os
import pandas as pd
import sys
import time

In [2]:
import arcpy

In [3]:

# ========= CONFIGURE THESE =========
region    = "KHMAO"
gdb_path  = r"C:\Users\User\Documents\ЦГД\KHMAO_RiskProfile_30-09-2025\KHMAO_RiskProfile\Default.gdb"
RISKS_FDS = "risk"   # feature dataset that holds risk layers
EXCEL_PATH = r"C:\Users\User\Documents\ЦГД\Структура базы_v03.xlsx"   # <-- put your local path here
SHEET_INDEX = 1       # 0=Economics, 1=Risks
DRY_RUN     = False   # True → only print, False → apply AlterField
VERBOSE_SKIPS = False # True → print per-field skips (can be very chatty)
LOG_EVERY   = 25      # print a heartbeat every N fields inside a FC

# ========= UTILS =========
def print_flush(*args, **kwargs):
    """print + flush so you see progress immediately in VS Code/terminal."""
    print(*args, **kwargs)
    sys.stdout.flush()

def secs(dt):
    """Format seconds nicely."""
    return f"{dt:.2f}s"

# ========= READ ALIAS MAP FROM EXCEL =========
t0 = time.time()
print_flush(f"[1/6] Reading Excel: {EXCEL_PATH} (sheet index={SHEET_INDEX}) ...")
df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_INDEX)
print_flush(f"       Rows read: {len(df)}; Columns: {list(df.columns)}")

# Normalize headers for flexible detection
hdr_map = {c: str(c).strip().lower() for c in df.columns}
df.columns = [hdr_map[c] for c in df.columns]

# Candidate columns for field names and aliases
name_candidates  = ["field_name_tot", "название поля", "name"]
alias_candidates = ["псевдонимы", "псевдоним", "alias", "field_alias"]

def pick_col(cands, cols):
    for c in cands:
        if c in cols:
            return c
    raise ValueError(f"Не найден столбец среди вариантов: {cands} | имеющиеся: {list(cols)}")

name_col  = pick_col(name_candidates,  df.columns)
alias_col = pick_col(alias_candidates, df.columns)
print_flush(f"       Detected columns -> name: '{name_col}', alias: '{alias_col}'")

# Build mapping: exact field name (as in GDB) -> alias
df = df[[name_col, alias_col]].dropna()
df[name_col]  = df[name_col].astype(str).str.strip()
df[alias_col] = df[alias_col].astype(str).str.strip()

alias_map_exact = dict(zip(df[name_col], df[alias_col]))
alias_map_lower = {k.lower(): v for k, v in alias_map_exact.items()}
print_flush(f"       Alias rows usable: {len(alias_map_exact)} "
            f"(example: {list(alias_map_exact.items())[:3]})")
print_flush(f"       Excel read & mapping built in {secs(time.time()-t0)}")

# ========= FIND FEATURE CLASSES IN 'risk' =========
arcpy.env.workspace = os.path.join(gdb_path, RISKS_FDS)
fcs = arcpy.ListFeatureClasses() or []
print_flush(f"\n[2/6] Scanning feature dataset: {RISKS_FDS}")
print_flush(f"       Feature classes found: {len(fcs)}")
if fcs:
    preview = ", ".join(fcs[:5])
    print_flush(f"       First few FCs: {preview}{' ...' if len(fcs) > 5 else ''}")

# ========= APPLY ALIASES =========
print_flush(f"\n[3/6] Starting alias updates (DRY_RUN={DRY_RUN}) ...")

changes = []   # (fc, field_name, old_alias, new_alias, duration_s)
skipped = []   # (fc, field_name, reason)

grand_total_fields = 0
grand_to_change    = 0

for idx, fc in enumerate(fcs, start=1):
    if "template" in fc.lower():
        skipped.append((fc, None, "template fc skipped"))
        continue

    fc_path = os.path.join(gdb_path, RISKS_FDS, fc)
    print_flush(f"\n[FC {idx}/{len(fcs)}] {fc} -> {fc_path}")
    t_fc = time.time()

    try:
        fields = arcpy.ListFields(fc_path)
    except Exception as e:
        print_flush(f"       ERROR: cannot ListFields -> {e}")
        skipped.append((fc, None, f"ListFields error: {e}"))
        continue

    # exclude OID/Geometry
    data_fields = [f for f in fields if f.type not in ("OID", "Geometry")]
    print_flush(f"       Fields (excl. OID/Geometry): {len(data_fields)}")
    grand_total_fields += len(data_fields)

    to_change = 0
    processed = 0

    for i, f in enumerate(data_fields, start=1):
        processed += 1
        # heartbeat
        if (processed % LOG_EVERY) == 0:
            print_flush(f"         ...processed {processed}/{len(data_fields)} fields")

        desired = alias_map_exact.get(f.name) or alias_map_lower.get(f.name.lower())
        if not desired:
            if VERBOSE_SKIPS:
                print_flush(f"         SKIP (no alias in Excel): {f.name} | current='{f.aliasName}'")
            skipped.append((fc, f.name, "alias not found in Excel"))
            continue

        current_alias = f.aliasName or ""
        if current_alias == desired:
            if VERBOSE_SKIPS:
                print_flush(f"         OK (already correct): {f.name} -> '{current_alias}'")
            skipped.append((fc, f.name, "already correct"))
            continue

        to_change += 1
        t_field = time.time()
        msg = f"         CHANGE: {f.name}: '{current_alias}' -> '{desired}'"
        if DRY_RUN:
            print_flush(msg + " [dry-run]")
            changes.append((fc, f.name, current_alias, desired, 0.0))
            continue

        print_flush(msg + " ... applying")
        try:
            arcpy.management.AlterField(
                in_table=fc_path,
                field=f.name,
                new_field_name=None,        # keep actual name
                new_field_alias=desired,    # set alias
                field_type=None,
                field_length=None,
                field_is_nullable=None,
                clear_field_alias=False
            )
            dur = time.time() - t_field
            print_flush(f"           ✔ done in {secs(dur)}")
            changes.append((fc, f.name, current_alias, desired, dur))
        except Exception as e:
            print_flush(f"           ✖ ERROR: {e}")
            skipped.append((fc, f.name, f"ERROR: {e}"))

    dur_fc = time.time() - t_fc
    grand_to_change += to_change
    print_flush(f"       FC summary: processed={processed}, changed={to_change}, "
                f"skipped={processed - to_change} | took {secs(dur_fc)}")

# ========= REPORT =========
print_flush(f"\n[4/6] Summary across all FCs in '{RISKS_FDS}'")
print_flush(f"       Total fields seen: {grand_total_fields}")
print_flush(f"       Total fields changed: {grand_to_change}")
print_flush(f"       Total skipped: {len(skipped)}")

# breakdown of skip reasons (top few)
reason_counts = {}
for _, _, r in skipped:
    reason_counts[r] = reason_counts.get(r, 0) + 1
if reason_counts:
    print_flush("       Skip reasons:")
    for r, n in sorted(reason_counts.items(), key=lambda x: -x[1])[:10]:
        print_flush(f"         {r}: {n}")

# optional: print the first few changes
if changes:
    print_flush("\n[5/6] Example changes (first 10):")
    for rec in changes[:10]:
        fc, fld, old, new, dur = rec
        print_flush(f"       [{fc}] {fld}: '{old}' -> '{new}' in {secs(dur)}")

print_flush(f"\n[6/6] DONE. DRY_RUN={DRY_RUN}.")

[1/6] Reading Excel: C:\Users\User\Documents\ЦГД\Структура базы_v03.xlsx (sheet index=1) ...
       Rows read: 66; Columns: ['Группа полей', 'БДv01\nПоле шейпа (10 симв., латынь)', 'field_name', 'field_name_tot', 'Тип данных', 'alias', 'Размерность', 'Метео ОПЯ?', 'Неметео ОПЯ?', 'Описание', 'Откуда берутся данные', 'Если есть modelBuilder']
       Detected columns -> name: 'field_name_tot', alias: 'alias'
       Alias rows usable: 63 (example: [('region', 'Регион'), ('MO', 'МО'), ('area', 'Площадь общая')])
       Excel read & mapping built in 1.10s

[2/6] Scanning feature dataset: risk
       Feature classes found: 11
       First few FCs: KHMAO_hazHeat_MO, KHMAO_hazardTemplate_MO, KHMAO_hazCold_MO, KHMAO_hazRain_MO, KHMAO_hazThunderstorm_MO ...

[3/6] Starting alias updates (DRY_RUN=False) ...

[FC 1/11] KHMAO_hazHeat_MO -> C:\Users\User\Documents\ЦГД\KHMAO_RiskProfile_30-09-2025\KHMAO_RiskProfile\Default.gdb\risk\KHMAO_hazHeat_MO
       Fields (excl. OID/Geometry): 67
         CHAN

In [5]:
ECON_FDS     = "economics"   # feature dataset that holds Economics layers
SHEET_INDEX  = 0             # 0 = Economics (as you said)
DRY_RUN      = False         # True → only print; False → apply AlterField
VERBOSE_SKIPS = False        # True → print per-field skips
LOG_EVERY    = 25            # heartbeat every N fields

# ========= UTILS =========
def print_flush(*args, **kwargs):
    print(*args, **kwargs); sys.stdout.flush()

def secs(dt): return f"{dt:.2f}s"

def pick_col(cands, cols):
    for c in cands:
        if c in cols:
            return c
    raise ValueError(f"Не найден столбец среди вариантов: {cands} | имеющиеся: {list(cols)}")

# ========= READ ALIAS MAP FROM EXCEL (SHEET 0 = ECONOMICS) =========
t0 = time.time()
print_flush(f"[1/6] Reading Excel: {EXCEL_PATH} (sheet index={SHEET_INDEX}) ...")
df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_INDEX)
print_flush(f"       Rows read: {len(df)}; Columns: {list(df.columns)}")

# Normalize headers for flexible detection
hdr_map = {c: str(c).strip().lower() for c in df.columns}
df.columns = [hdr_map[c] for c in df.columns]

# Candidate columns for field names and aliases
name_candidates  = [ "field_name_tot", "название поля", "name"]
alias_candidates = ["псевдонимы", "псевдоним", "alias", "field_alias"]

name_col  = pick_col(name_candidates,  df.columns)
alias_col = pick_col(alias_candidates, df.columns)
print_flush(f"       Detected columns -> name: '{name_col}', alias: '{alias_col}'")

# Build mapping: exact field name (as in GDB) -> alias
df = df[[name_col, alias_col]].dropna()
df[name_col]  = df[name_col].astype(str).str.strip()
df[alias_col] = df[alias_col].astype(str).str.strip()

alias_map_exact = dict(zip(df[name_col], df[alias_col]))
alias_map_lower = {k.lower(): v for k, v in alias_map_exact.items()}

print_flush(f"       Alias rows usable: {len(alias_map_exact)} "
            f"(example: {list(alias_map_exact.items())[:3]})")
print_flush(f"       Excel read & mapping built in {secs(time.time()-t0)}")

# ========= FIND FEATURE CLASSES IN 'economics' =========
arcpy.env.workspace = os.path.join(gdb_path, ECON_FDS)
fcs = arcpy.ListFeatureClasses() or []
print_flush(f"\n[2/6] Scanning feature dataset: {ECON_FDS}")
print_flush(f"       Feature classes found: {len(fcs)}")
if fcs:
    preview = ", ".join(fcs[:5])
    print_flush(f"       First few FCs: {preview}{' ...' if len(fcs) > 5 else ''}")

# ========= APPLY ALIASES =========
print_flush(f"\n[3/6] Starting alias updates (DRY_RUN={DRY_RUN}) ...")

changes = []   # (fc, field_name, old_alias, new_alias, duration_s)
skipped = []   # (fc, field_name, reason)

grand_total_fields = 0
grand_to_change    = 0

for idx, fc in enumerate(fcs, start=1):
    # optional: skip templates if any
    if "template" in fc.lower():
        skipped.append((fc, None, "template fc skipped"))
        continue

    fc_path = os.path.join(gdb_path, ECON_FDS, fc)
    print_flush(f"\n[FC {idx}/{len(fcs)}] {fc} -> {fc_path}")
    t_fc = time.time()

    try:
        fields = arcpy.ListFields(fc_path)
    except Exception as e:
        print_flush(f"       ERROR: cannot ListFields -> {e}")
        skipped.append((fc, None, f"ListFields error: {e}"))
        continue

    # exclude OID/Geometry
    data_fields = [f for f in fields if f.type not in ("OID", "Geometry")]
    print_flush(f"       Fields (excl. OID/Geometry): {len(data_fields)}")
    grand_total_fields += len(data_fields)

    to_change = 0
    processed = 0

    for i, f in enumerate(data_fields, start=1):
        processed += 1
        # heartbeat
        if (processed % LOG_EVERY) == 0:
            print_flush(f"         ...processed {processed}/{len(data_fields)} fields")

        desired = alias_map_exact.get(f.name) or alias_map_lower.get(f.name.lower())
        if not desired:
            if VERBOSE_SKIPS:
                print_flush(f"         SKIP (no alias in Excel): {f.name} | current='{f.aliasName}'")
            skipped.append((fc, f.name, "alias not found in Excel"))
            continue

        current_alias = f.aliasName or ""
        if current_alias == desired:
            if VERBOSE_SKIPS:
                print_flush(f"         OK (already correct): {f.name} -> '{current_alias}'")
            skipped.append((fc, f.name, "already correct"))
            continue

        to_change += 1
        t_field = time.time()
        msg = f"         CHANGE: {f.name}: '{current_alias}' -> '{desired}'"
        if DRY_RUN:
            print_flush(msg + " [dry-run]")
            changes.append((fc, f.name, current_alias, desired, 0.0))
            continue

        print_flush(msg + " ... applying")
        try:
            arcpy.management.AlterField(
                in_table=fc_path,
                field=f.name,
                new_field_name=None,        # keep actual name
                new_field_alias=desired,    # set alias
                field_type=None,
                field_length=None,
                field_is_nullable=None,
                clear_field_alias=False
            )
            dur = time.time() - t_field
            print_flush(f"           ✔ done in {secs(dur)}")
            changes.append((fc, f.name, current_alias, desired, dur))
        except Exception as e:
            print_flush(f"           ✖ ERROR: {e}")
            skipped.append((fc, f.name, f"ERROR: {e}"))

    dur_fc = time.time() - t_fc
    grand_to_change += to_change
    print_flush(f"       FC summary: processed={processed}, changed={to_change}, "
                f"skipped={processed - to_change} | took {secs(dur_fc)}")

# ========= REPORT =========
print_flush(f"\n[4/6] Summary across all FCs in '{ECON_FDS}'")
print_flush(f"       Total fields seen: {grand_total_fields}")
print_flush(f"       Total fields changed: {grand_to_change}")
print_flush(f"       Total skipped: {len(skipped)}")

# breakdown of skip reasons (top few)
reason_counts = {}
for _, _, r in skipped:
    reason_counts[r] = reason_counts.get(r, 0)


[1/6] Reading Excel: C:\Users\User\Documents\ЦГД\Структура базы_v03.xlsx (sheet index=0) ...
       Rows read: 48; Columns: ['Группа полей', 'field_name', 'field_name_tot', 'Тип данных', 'alias', 'Размерность', 'Описание', 'Откуда берутся данные', 'Если есть modelBuilder']
       Detected columns -> name: 'field_name_tot', alias: 'alias'
       Alias rows usable: 47 (example: [('region', 'Регион'), ('MO', 'МО'), ('area', 'Площадь общая')])
       Excel read & mapping built in 0.09s

[2/6] Scanning feature dataset: economics
       Feature classes found: 5
       First few FCs: KHMAO_houses_src, KHMAO_forest_src, KHMAO_agriculture_src, KHMAO_roads_src, KHMAO_economics_MO

[3/6] Starting alias updates (DRY_RUN=False) ...

[FC 1/5] KHMAO_houses_src -> C:\Users\User\Documents\ЦГД\KHMAO_RiskProfile_30-09-2025\KHMAO_RiskProfile\Default.gdb\economics\KHMAO_houses_src
       Fields (excl. OID/Geometry): 11
         CHANGE: region: 'region' -> 'Регион' ... applying
           ✔ done in 0.60s
  